In [ ]:

import datetime
import hashlib
import json
from flask import Flask, jsonify, request
import requests
from uuid import uuid4
from urllib.parse import urlparse

 # Blockchain creation

class Blockchain:

    def __init__(self):
        self.chain = []
        self.transactions = []
        self.genisus_block(proof = 1, previous_hash ='0')
        self.nodes = set()

    def genisus_block(self, proof, previous_hash):
        block = {'index': len(self.chain) + 1,
                 'timestamp': str(datetime.datetime.now()),
                 'proof': proof,
                 'previous_hash': previous_hash,
                 'transactions': self.transactions}
        self.transactions = []
        self.chain.append(block)
        return block

    def get_last_block(self):
        return self.chain[-1]

    def proof_of_work(self, previous_proof):
        new_proof = 1
        check_proof = False
        while check_proof is False:
            hash_operation = hashlib.sha256(str(new_proof**2 - previous_proof**2).encode()).hexdigest()
            if hash_operation[:4] == '0000':
                check_proof = True
            else:
                new_proof += 1
        return new_proof

    def hash(self, block):
        encoded_block = json.dumps(block, sort_keys = True).encode()
        return hashlib.sha256(encoded_block).hexdigest()

    def check_chain_validity(self, chain):
        last_block = chain[0]
        block_index = 1
        while block_index < len(chain):
            block = chain[block_index]
            if block['previous_hash'] != self.hash(last_block):
                return False
            previous_proof = last_block['proof']
            proof = block['proof']
            hash_operation = hashlib.sha256(str(proof**2 - previous_proof**2).encode()).hexdigest()
            if hash_operation[:4] != '0000':
                return False
            last_block = block
            block_index += 1
        return True

    def add_transaction(self, sender, receiver, amount):
        self.transactions.append({'sender': sender,
                                  'receiver': receiver,
                                  'amount': amount})
        last_block = self.get_last_block()
        return last_block['index'] + 1

# Part 2 - Mining our Blockchain

# Creating a Web App
app = Flask(__name__)

# Creating an address for the node on Port 5001
node_address = str(uuid4()).replace('-', '')

# Creating a Blockchain
blockchain = Blockchain()

# Mining a new block
@app.route('/mine_block', methods = ['GET'])
def mine_block():
    last_block = blockchain.get_last_block()
    previous_proof = last_block['proof']
    proof = blockchain.proof_of_work(previous_proof)
    last_block_hash = blockchain.hash(last_block)
    blockchain.add_transaction(sender = node_address, receiver = 'Prateek', amount = 100000)
    block = blockchain.genisus_block(proof, last_block_hash)
    response = {'message': 'Congratulations, you just mined a block!',
                'index': block['index'],
                'timestamp': block['timestamp'],
                'proof': block['proof'],
                'last_block_hash': block['previous_hash'],
                'transactions': block['transactions']}
    return jsonify(response), 200

# Getting the full Blockchain
@app.route('/get_chain', methods = ['GET'])
def get_chain():
    response = {'chain': blockchain.chain,
                'length': len(blockchain.chain)}
    return jsonify(response), 200

# Checking if the Blockchain is valid
@app.route('/is_valid', methods = ['GET'])
def is_valid():
    is_valid = blockchain.check_chain_validity(blockchain.chain)
    if is_valid:
        response = {'message': 'Blockchain is perfectly valid.'}
    else:
        response = {'message': 'The Blockchain is not valid.'}
    return jsonify(response), 200

# Running the app
app.run(host = '0.0.0.0', port = 5001)


 * Running on http://0.0.0.0:5001/ (Press CTRL+C to quit)
127.0.0.1 - - [06/Apr/2019 11:10:32] "GET /get_chain HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2019 11:10:40] "GET /mine_block HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2019 11:10:42] "GET /mine_block HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2019 11:10:43] "GET /mine_block HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2019 11:10:53] "GET /is_valid HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2019 11:11:00] "GET /get_chain HTTP/1.1" 200 -
